In [41]:
# This is example code for quering the Infra using LLM
# This code is referred in below blog post
# 

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install csvkit

In [17]:
PROMPT = """
You are a PSQL query generator expert. Below is the table information

Table Name: ec2_instances
Schema:
      Column        |       Type        
--------------------+-------------------
 InstanceId      | character varying 
 InstanceType    | character varying 
 State            | character varying 
 LaunchTime      | character varying 
 PrivateIP       | character varying 
 PublicIP        | character varying 
 SubnetId        | character varying 
 VPCId           | character varying 
 InstanceProfile | character varying    
 SecurityGroups   | character varying 

Task: Based on the user question generate the psql query
Instructions:
 - Map the user question with one of the Column value which is relavent
 - Use only the provided Column names mentioned in the above schema 
 - Query should get what is needed
 
The Answer should contain only query no extra content. The question as follows
"""

In [21]:
import subprocess 

def csv_output(query):
    """
        Execute the SQL query on csv file using csvsql package
    """
    csv_file_path = "ec2_instances.csv"

    result = subprocess.run(['csvsql', '--query', query, csv_file_path], stdout=subprocess.PIPE, text=True)

    return result.stdout
    

In [18]:
import os

os.environ['OPENAI_API_KEY'] = 'YOUR_AWESOME_OPEN_API_KEY'

In [29]:
USER_QUESTION= "How many instances are there with m5.xlarge instance type"

In [30]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
        "role": "system", 
        "content": PROMPT
    },
    {
        "role": "user", 
        "content": USER_QUESTION
    }
  ]
)

print(completion.choices[0].message.content)

SELECT COUNT(InstanceId) 
FROM ec2_instances 
WHERE InstanceType = 'm5.xlarge';


In [31]:
query = completion.choices[0].message.content

output = csv_output(query)

print(output)

COUNT(InstanceId)
8



In [36]:
# 
# This is optional.
# With this prompt we show the Output as Chat Response
# 

PROMPT_FOR_FINAL_OUTPUT = """
You are expert in understanding the SQL query output which is generated by python code. 

You will be given question asked by the user and output generated by the sql query. 

Based on the that give me the chat response to user
"""

USER_QUESTION_WITH_ANWSERS = f"""
user question: {USER_QUESTION}
output: {output}
"""

In [37]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
        "role": "system", 
        "content": PROMPT_FOR_FINAL_OUTPUT
    },
    {
        "role": "user", 
        "content": USER_QUESTION_WITH_ANWSERS
    }
  ]
)


print(completion.choices[0].message.content)

There are 8 instances with m5.xlarge instance type.
